In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.optimizers import Adadelta
import numpy as np
import requests
import gzip



def get_testset():
    url = 'https://courses.engr.illinois.edu/ece498icc/sp2020/lab1_request_dataset.php'
    values = {'request': 'testdata', 'netid':'myun7'}
    r = requests.post(url, data=values, allow_redirects=True)
    filename = r.url.split("/")[-1]
    testset_id = filename.split(".")[0].split("_")[-1]
    with open(filename, 'wb') as f: 
        f.write(r.content)
    return load_dataset(filename), testset_id

def load_dataset(path):
    num_img = 1000
    with gzip.open(path, 'rb') as infile:
        data = np.frombuffer(infile.read(), dtype=np.uint8).reshape(num_img, 784)
    return data

def create_model():
    model = Sequential()
    #add model layers
    model.add(Conv2D(3, kernel_size=5, strides = 1, activation='relu', input_shape=(28,28,1)))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(Conv2D(3, kernel_size=3, strides = 1, padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adadelta(),loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a basic model instance
model = create_model()
checkpoint_path = "training_1/cp.ckpt"

# Loads the weights 
model.load_weights(checkpoint_path)


data, testset_id  = get_testset()
data = data.reshape(data.shape[0], 28, 28, 1)
testset_id


predictions = model.predict(data)
prediction = ''
for i in predictions:
    prediction += str(np.argmax(i))
url = 'https://courses.engr.illinois.edu/ece498icc/sp2020/lab1_request_dataset.php'
values = {'request': 'verify', 'netid':'myun7', 'testset_id':testset_id,'prediction':prediction}
r = requests.post(url, data=values, allow_redirects=True)
print(r.text)


865


In [14]:
testset_id

'5832ba5e'

In [18]:
print(tf.__version__)

1.14.0
